# Upload Trove newspaper articles to Omeka-S

I was keen to play around with [Omeka-S](https://omeka.org/s/) – the new, Linked Open Data powered version of Omeka. In particular I wanted to understand its [API](https://omeka.org/s/docs/developer/key_concepts/api/), its use of [JSON-LD](https://json-ld.org/), and how I could create and use relationships between different types of entities. I decided that one way of exploring these things was to build a pipeline that would make it easy to upload Trove newspaper articles to an Omeka site. I recently figured out how to generate nice thumbnails from newspaper articles, so I could add images as well. But what about the 'Linked' part of LOD? I thought I'd not only create records for the newspaper articles, I'd create records for each cited newspaper and link them to the articles. Things not strings!

Of course, this is not the only way to get Trove records into Omeka. You can save them to [Zotero](https://www.zotero.org/) and then use the [Zotero Import module](https://omeka.org/s/docs/user-manual/modules/zoteroimport/) to upload to Omeka. My [first Omeka 'how to'](http://discontents.com.au/some-exhibition-magic-with-zotero-and-omeka/) (written 8 years ago!) talked about using the Zotero Import plugin to upload records from the National Archives of Australia. While the import module works well, I wanted the flexibility to generate new images on the fly, modify the metadata, and unflatten the Zotero records to build newspaper-to-article relationships. I also wanted to be able to get newspaper articles from other sources, such as Trove lists.

As I started playing around I realised I could rejig my thumbnail generator to [get an image of the whole article](https://nbviewer.jupyter.org/github/GLAM-Workbench/trove-newspapers/blob/master/Save-Trove-newspaper-article-as-image.ipynb). This is useful because Trove's article download options tend to slice up images in unpleasant ways. So I decided to upload the complete image (or images if the article is published across multiple pages) and let Omeka create the derivative versions.

In summary, you can use this notebook to:

* Get a Trove newspaper article using the Trove API
* Generate an image for the article
* Search the Omeka site using the API to see if the newspaper that published the article already has a record, if not create one.
* Upload the article details to Omeka using the API, including a link to the newspaper record, as well as the article image(s).

But what newspaper articles are we uploading? I've created four different ways of selecting articles for upload. There are further details and configuration options for each of these  methods below:

* [Option 1: Upload a Trove newspaper search](#Option-1%3A-Upload-a-Trove-newspaper-search)
* [Option 2: Upload newspaper articles from a Trove list](#Option-2%3A-Upload-newspaper-articles-from-a-Trove-list)
* [Option 3: Upload Trove newspaper articles saved in Zotero](#Option-3%3A-Upload-Trove-newspaper-articles-saved-in-Zotero)
* [Option 4: Upload a list of article ids](#Option-4%3A-Upload-a-list-of-article-ids)

There's a fair bit of configuration involved in getting this all working, so make sure you follow the instructions under [Basic configuration](#Basic-configuration) and [Preparing Omeka-S](#Preparing-Omeka-S) before proceeding.

## An example

Here's an example of a [newspaper article](https://trove.nla.gov.au/newspaper/article/162833980) in Trove:

<img src="images/162833980.png" style="width: 600px;">

Here's the same article after it's been imported into Omeka. Note that the article is linked to the newspaper that published it using the `isPartOf` property. You can also see the images and PDF of the article that were automatically uploaded along with the metadata.

<img src="images/article_omeka.png" style="width: 600px;">

And here's the record for the linked newspaper:

<img src="images/newspaper_omeka.png" style="width: 600px;">


## Basic configuration

First of all you need an Omeka-S site to upload to! If you don't have a site, a handy server, or a preferred web host, I'd suggest you set up an account with [Reclaim Hosting](https://reclaimhosting.com/) – they're affordable, have excellent support, and provide a one-click installer for Omeka-S. You'll be up and running very quickly.

Once you have your Omeka-S site, you need to add some configuration values in the cell below:

* `API_URL` – the url (or endpoint) of your Omeka site's API. This is basically just the url of your site with `/api` on the end.
* `KEY_IDENTITY` and `KEY_CREDENTIAL` these are the authentication keys you need to upload new records to Omeka-S.
* `TROVE_API_KEY` – authentication key to access the Trove API

See below for instructions on generating your keys.

### Generating your Omeka keys

1. To generate your keys, log in to the admin interface of your Omeka-S site and click on 'Users' in the side menu. 
2. Find your user account in the list and then click on the pencil icon to edit your details. 
3. Click on the 'API keys' tab.
4. In the 'New key label' box, enter a name for your key – something like 'Trove upload' would be fine.
5. Click on the 'Save' button.
6. A message will appear with your `key_identity` and `key_credential` values – the `key_credential` is only ever displayed once, so copy them both now!

### Trove API key

To get your Trove API key, just [follow these instructions](http://help.nla.gov.au/trove/building-with-trove/api).


## Preparing Omeka-S

Before we start uploading newspaper articles we need to set up the vocabularies and templates we need in Omeka to describe them.

### Installing the schema.org vocabulary

Omeka uses defined vocabularies to describe items. A number of vocabularies are built-in, but we're going to add the widely-used [Schema.org](https://schema.org/) vocabularly. Amongst many other things, Schema.org includes classes for both `Newspaper` and `NewsArticle`. You can download the [Schema.org definition files here](https://schema.org/docs/developers.html). In the instructions below, I suggest installing the 'all layers' version of the definition file, as this includes the [Bib extension](https://bib.schema.org/) as well as [Pending changes](https://pending.schema.org/) such as `ArchiveComponent` and `ArchiveOrganization`.

To install Schema.org in Omeka:

1. Log in to the admin interface of your Omeka-S site and click on 'Vocabularies' in the side menu.
2. Click on the 'Import new vocabulary button.
3. In the `Vocabularly url` field enter http://schema.org/version/latest/all-layers.rdf
4. In the `File format` field select 'RDF/XML'
5. In the `Prefix` field enter 'schema'
6. In the `Namespace URI` field enter http://schema.org/
7. In the `Label` field enter 'Schema.org'
8. Click 'Import' to complete the installation.

This is what the import form should look like:

<img src="images/omeka-vocab-import.png" width="600">

See the Omeka-S documentation for [more information on managing vocabularies](https://omeka.org/s/docs/user-manual/content/vocabularies/).

### Installing the Numeric Data Type module

The [Numeric Data Type module](https://omeka.org/s/docs/user-manual/modules/numericdatatypes/) gives you more options in defining the data type of a field. In particular, you can identify certain values as ISO formatted dates so they can then be properly formatted, sorted, and searched. The template I've created for newspaper articles (see below) uses the 'timestamp' data type to import the dates of the newspaper articles, so you'll need to install this module before importing the templates.

1. Download the [Numeric Data Type module](https://omeka.org/s/modules/NumericDataTypes/) to your computer.
2. Upload the zipped module to the modules folder of your Omeka site.
3. Unzip the module.
4. Log in to the admin interface of your Omeka-S site and click on 'Modules' in the side menu.
5. Click on the 'Install' button.

See the Omeka-S documentation for [more information on installing modules](https://omeka.org/s/docs/user-manual/modules/#installing-modules).

### Importing the resource templates 

One of the powerful features of Omeka is the ability to define the types of items you're working with using 'resource templates'. These resource templates associate the items with specific vocabulary classes and list the expected properties. I've created resource templates for 'Newspaper' and 'Newspaper article' and exported them as JSON. The Trove upload code looks for these templates, so they need to be imported into your Omeka site.

1. Download [Newspaper.json](templates/Newspaper.json) and [Newspaper_article.json](templates/Newspaper_article.json) to your own computer.
2. Log in to the admin interface of your Omeka-S site and click on 'Resource templates' in the side menu.
3. Click on the 'Import' button
4. Click on 'Browse' and select the `Newspaper.json` file you downloaded.
5. Click on the 'Review import' button.
6. Click on the 'Complete import' button.
7. Click the 'Edit Resource Template' button.
8. Find the `name` property and click on the pencil icon to edit it.
9. Check the box next to 'Use for resource title', then click on the 'Set changes' button.
10. Find the `description` property and click on the pencil icon to edit it.
11. Check the box next to 'Use for resource description', then click on the 'Set changes' button.
12. Click on the 'Save' button.
13. Repeat the same procedure to import `Newspaper_article.json`, but check at the 'Review import' stage to make sure that the selected data type for the `datePublished` property is `Timestamp`.

Note that I've deliberately kept the templates fairly simple. You might want to add additional properties. However, if you change or remove any of the existing properties, you'll probably also have to edit the `add_article()` function below.

See the Omeka-S documentation for [more information on managing resource templates](https://omeka.org/s/docs/user-manual/content/resource-template/).

## Define all the functions that we need

Hit **Shift+Enter** to run each of the cells below and set up all the basic configuration and functions we need.

In [ ]:
import os
import re
from pathlib import Path

import arrow
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from omeka_s_tools.api import OmekaAPIClient
from pyzotero import zotero
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from tqdm.auto import tqdm
from trove_newspaper_images.articles import download_images

s = requests.Session()
retries = Retry(total=10, backoff_factor=1, status_forcelist=[502, 503, 504, 524])
s.mount("http://", HTTPAdapter(max_retries=retries))
s.mount("https://", HTTPAdapter(max_retries=retries))

load_dotenv()

Insert the values of your Trove API key and your Omeka API url and authentication keys below.

In [ ]:
# CONFIGURATION
# Things you need to change!
# Paste your values in below!

# The url of your Omeka site's api (basically your Omeka site site url with '/api' on the end)
API_URL = "http://your.omeka.site/api"

# The keys to your Omeka site
KEY_IDENTITY = "YOUR OMEKA KEY IDENTITY"
KEY_CREDENTIAL = "YOUR OMEKA KEY CREDENTIAL"

# Your Trove API key
TROVE_API_KEY = "YOUR TROVE API KEY"

# Alternatively, use api keys and settings from environment variables if available
if os.getenv("TROVE_API_KEY"):
    TROVE_API_KEY = os.getenv("TROVE_API_KEY")
if os.getenv("OMEKA_KEY_IDENTITY"):
    KEY_IDENTITY = os.getenv("OMEKA_KEY_IDENTITY")
if os.getenv("OMEKA_KEY_CREDENTIAL"):
    KEY_CREDENTIAL = os.getenv("OMEKA_KEY_CREDENTIAL")
if os.getenv("OMEKA_API_URL"):
    API_URL = os.getenv("OMEKA_API_URL")

TROVE_HEADERS = {"X-API-KEY": TROVE_API_KEY}

# Resize images so this is the max dimension -- the Trove page images are very big, so you might want to resize before uploading to Omeka
# Set this to None if you want them as big as possible (this might be useful if you're using the Omeka IIIF server & Universal viewer modules)
MAX_IMAGE_SIZE = 3000

In [ ]:
omeka = OmekaAPIClient(
    API_URL, key_identity=KEY_IDENTITY, key_credential=KEY_CREDENTIAL
)


def get_article(article_id):
    """
    Retrieve an individual newspaper article from the Trove API.

    Parameters:
    * `article_id` - a Trove article identifier

    Returns:
    * a dict with article metadata from Trove API
    """
    url = "http://api.trove.nla.gov.au/v3/newspaper/{}".format(article_id)
    params = {"include": "articleText", "encoding": "json"}
    response = s.get(url, params=params, headers=TROVE_HEADERS)
    return response.json()


def check_for_item(item_url, template_id):
    """
    Check to see if an item exists in Omeka using schema:url values and template ids.

    Parameters:
    * `item_url` - a unique url saved in the schema:url field of an item
    * `template_id` - the Omeka id of a template to filter item results

    Returns:
    * the JSON-LD item representation if found, None if not.
    """
    # Filter items by the supplied parameters
    results = omeka.filter_items_by_property(
        filter_property="schema:url",
        filter_value=item_url,
        resource_template_id=template_id,
    )
    # Get the first record, or None if there are no matches
    try:
        item = results["results"][0]
    except (KeyError, IndexError):
        item = None
    return item


def add_newspaper(newspaper):
    """
    Check to see if the given newspaper has already been uploaded to Omeka.
    If not, upload metadata to Omeka.

    Parameters:
    * `newspaper` - this is the dict identifying the newspaper from a Trove article record

    Returns:
    * the Omeka id of the newspaper record
    """
    # Get details of the Newspaper template
    newspaper_template = omeka.get_template_by_label("Newspaper")
    template_id = newspaper_template["o:id"]
    # Get the class used with the Newspaper template
    class_id = newspaper_template["o:resource_class"]["o:id"]
    # Construct a Trove persistent url for the newspaper
    newspaper_url = f'http://nla.gov.au/nla.news-title{newspaper["id"]}'
    # Check to see if the newspaper has already been uploaded to Omeka
    newspaper_item = check_for_item(newspaper_url, template_id)
    # If it hasn't been uploaded, upload it!
    if not newspaper_item:
        # Prepare the data
        newspaper_data = {
            "schema:name": [newspaper["title"]],
            "schema:identifier": [newspaper["id"]],
            "schema:url": [newspaper_url],
        }
        # Construct the payload for upload to omeka
        payload = omeka.prepare_item_payload_using_template(newspaper_data, template_id)
        # Upload the item
        newspaper_item = omeka.add_item(
            payload, template_id=template_id, class_id=class_id
        )
    return newspaper_item["o:id"]


def add_article(article_id):
    """
    Check to see if the given article has already been uploaded to Omeka.
    If not, retrieve information about a newspaper article from Trove and
    upload the metadata, text, and images to Omeka.

    Parameters:
    * `article_id` - a Trove article identifier

    Returns:
    * a JSON-LD representation of the new Omeka item
    """
    # Get details of the newspaper article template
    article_template = omeka.get_template_by_label("Newspaper article")
    template_id = article_template["o:id"]
    # Get the resource class used with the newspaper article template
    class_id = article_template["o:resource_class"]["o:id"]
    # Construct a Trove article persistent url
    article_url = f"http://nla.gov.au/nla.news-article{article_id}"
    # Check to see if an article with this url has already been uploaded to Omeka
    article_item = check_for_item(article_url, template_id)
    # If the article hasn't been uploaded, we'll upload it!
    if not article_item:
        # Get article details from Trove
        article = get_article(article_id)
        # Format a description
        formatted_date = arrow.get(article["date"], "YYYY-MM-DD").format("D MMM YYYY")
        summary = (
            f'{formatted_date}, {article["title"]["title"]}, page {article["page"]}'
        )
        # Get the Omeka id of the newspaper it was published in
        newspaper_id = add_newspaper(article["title"])
        # Remove html tags from article text
        try:
            soup = BeautifulSoup(article["articleText"])
            article_text = soup.get_text()
        except KeyError:
            article_text = ""
        # Prepare the article metadata
        article_data = {
            "schema:name": [article["heading"]],
            "schema:description": [summary],
            "schema:datePublished": [article["date"]],
            "schema:isPartOf": [newspaper_id],
            "schema:pagination": [article["page"]],
            "schema:identifier": [article_id],
            "schema:url": [article_url],
            "schema:text": [article_text],
        }
        # Construct the payload for uploading to Omeka
        payload = omeka.prepare_item_payload_using_template(article_data, template_id)
        # Download images of the article
        article_images = download_images(article_id, output_dir="temp")
        image_paths = [Path("temp", i) for i in article_images]
        # Upload the article
        article_item = omeka.add_item(
            payload, media_files=image_paths, template_id=template_id, class_id=class_id
        )
    return article_item

## Select your upload method

Select the method you want to use to select and process your articles and follow the associated instructions.

----

### Option 1: Upload a Trove newspaper search

This will attempt to upload **all** the newspaper articles returned by a search to your Omeka site. Obviously you want to **restrict your search** to make sure you're only getting the articles you want – use things like facets and date ranges to keep the result set to a manageable size. **You have been warned...**

Once you've constructed your search, enter the various parameters below. You might want to add additional facets such as `l-decade` or `l-year`. Check out [Trove's API documentation](https://help.nla.gov.au/trove/building-with-trove/api-version-2-technical-guide) to see all the options.

Once you've edited the search details, hit **Shift+Enter** to run the cells below and start your upload.

In [ ]:
def get_total_results(params):
    """
    Get the total number of results for a Trove search.
    """
    these_params = params.copy()
    these_params["n"] = 0
    response = s.get(
        "https://api.trove.nla.gov.au/v3/result",
        params=these_params,
        headers=TROVE_HEADERS,
    )
    data = response.json()
    return int(data["category"][0]["records"]["total"])


def upload_trove_search(params):
    start = "*"
    total = get_total_results(params)
    with tqdm(total=total) as pbar:
        while start:
            params["s"] = start
            response = s.get(
                "https://api.trove.nla.gov.au/v3/result",
                params=params,
                headers=TROVE_HEADERS,
            )
            data = response.json()
            # The nextStart parameter is used to get the next page of results.
            # If there's no nextStart then it means we're on the last page of results.
            try:
                start = data["category"][0]["records"]["nextStart"]
            except KeyError:
                start = None
            for article in data["category"][0]["records"]["article"]:
                add_article(article["id"])
                pbar.update(1)

In [ ]:
# Edit/add search values and parameters as required. These are an example only!
trove_params = {
    "q": '"inigo jones"',  # required -- change to anything you might enter in the Trove search box (including double quotes for phrases and boolean operators like AND)
    "category": "newspaper",  # don't change this
    "l-artType": "newspaper",
    "l-illustrated": "true",  # edit or remove -- limits to illustrated articles
    "l-illtype": "Photo",  # edit or remove -- limits to illustrations with photos
    "l-word": "1000+ Words",  # edit or remove -- limits to article with more than 1000 words
    "include": "articleText",  # don't change this
    "encoding": "json",  # don't change this
}

upload_trove_search(trove_params)

----

### Option 2: Upload newspaper articles from a Trove list

You can upload any newspaper articles stored in a Trove list to your Omeka site. 

To find the `list_id`, just go to the list's web page. The `list_id` is the string of numbers that appears after 'id=' in the url. Once you have your `list_id`, paste it where indicated in the cell below.

Once you've edited the list details, hit **Shift+Enter** to run the cells below and start your upload.

In [ ]:
def upload_trove_list(list_id):
    """
    Upload any newspaper articles in the given Trove list to Omeka.
    """
    url = "http://api.trove.nla.gov.au/v3/list/{}".format(list_id)
    params = {"include": "listItems", "encoding": "json", "key": TROVE_API_KEY}
    response = s.get(url, params=params)
    data = response.json()
    for item in tqdm(data["listItem"]):
        for category, record in item.items():
            if category == "article":
                add_article(record["id"])

In [ ]:
# Paste the identifier of your list between the quotes
list_id = "[Your list ID]"
list_id = "83777"
upload_trove_list(list_id)

----

### Option 3: Upload Trove newspaper articles saved in Zotero

You can upload any Trove newspaper articles stored in a Zotero collection to your Omeka site.

To access Zotero you need four pieces of information:

* `ZOTERO_KEY` – generate an API for this application by [going here](https://www.zotero.org/settings/keys/new) once you've logged into the Zotero web site. Your key only need read access.
* `LIBRARY_ID` – a string of numbers that identifies your library. For personal libraries, go to [this page](https://www.zotero.org/settings/keys/) once you've logged into the Zotero web site and look for the line that says 'Your userID for use in API calls is...'. For group libraries, just open the groups web page – the `LIBRARY_ID` will be the string of numbers after '/groups/' in the url.
* `LIBRARY_TYPE` – either 'user' or 'group', depending on whether it's a personal library of a group library.
* `collection_id` – the id of the collection that contains your Trove newspaper articles. Just open the collection on the Zotero website – the `collection_id` is the string of letters and numbers that comes after '/collectionKey/' in the url.

For additional information see the [Pyzotero](https://github.com/urschrei/pyzotero) documention.

When you have all the values you need, simply paste them where indicated in the cells below.

Once you've added your details, hit **Shift+Enter** to run the cells below and start your upload.

In [ ]:
# ENTER YOUR VALUES BETWEEN THE QUOTES WHERE INDICATED
ZOTERO_KEY = "YOUR ZOTERO KEY"  # The Zotero API key you generated
LIBRARY_TYPE = "user"  # user or group
LIBRARY_ID = "YOUR ZOTERO ID"  # Either a personal user id or a group id

# Or you can store your information in a .env file
if os.getenv("ZOTERO_KEY"):
    ZOTERO_KEY = os.getenv("ZOTERO_KEY")
if os.getenv("ZOTERO_ID"):
    LIBRARY_ID = os.getenv("ZOTERO_ID")

In [ ]:
def upload_zotero_collection(coll_id):
    """
    Upload any Trove newspaper articles in the given collection to Omeka.
    """
    zot = zotero.Zotero(LIBRARY_ID, LIBRARY_TYPE, ZOTERO_KEY)
    items = zot.everything(zot.collection_items(coll_id))
    articles = []
    for item in items:
        # Filter out things that aren't newspaper articles
        try:
            url = item["data"]["url"]
            if (
                item["data"]["itemType"] == "newspaperArticle"
                and "nla.news-article" in url
            ):
                article_id = re.search(r"(\d+)$", url).group(1)
                articles.append(article_id)
        except KeyError:
            pass
    for article_id in tqdm(articles):
        add_article(article_id)

In [ ]:
# Paste your collection ID between the quotes below.
collection_id = "YOUR COLLECTION ID"
upload_zotero_collection(collection_id)

---- 

### Option 4: Upload a list of article ids

This is useful for testing – just get some Trove newspaper article identifiers (the number in the url) and upload them to your Omeka site.

Once you've edited the list of article, hit **Shift+Enter** to run the cells below and start your upload.

In [ ]:
# Edit the list of articles as you see fit...
article_ids = [130413505, 65179201]

for article_id in tqdm(article_ids):
    add_article(article_id)

## Future developments

* I'd originally hoped to use the Omeka-S API to do most of the configuration, including importing vocabularies and resource templates. But I [just couldn't get it to work properly](https://forum.omeka.org/t/upload-resource-template-via-api/8855/5). If I figure it out I'll add the details.

* It would be good to be able to upload other types of things from Trove such as images, maps and journal articles. This should be possible, however, things are not as standardised as they are with the newspaper articles, so it'll take a bit of work. Let me know if you're interested and it'll give me a bit more motivation.

* Also, going back to my Omeka how-to from 8 years back, I'll be creating a RecordSearch to Omeka-S pipeline as well.

----

Created by [Tim Sherratt](https://timsherratt.org/) for the [GLAM Workbench](https://glam-workbench.github.io/).  
Support this project by becoming a [GitHub sponsor](https://github.com/sponsors/wragge?o=esb).
